<a href="https://colab.research.google.com/github/Spandana2704/ML/blob/main/ML(W4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv('/content/titanic_train.csv')
df.shape

(891, 12)

In [8]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
df=df.drop(['PassengerId','Name'],axis=1)

In [10]:
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
[col for col in df.columns if df[col].isnull().sum()>0]

['Age', 'Cabin', 'Embarked']

In [12]:
df['Age'] = df['Age'].fillna(df['Age'].median())

In [13]:
df=df.drop(['Embarked','Cabin'],axis=1)

In [14]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [15]:
from sklearn.metrics import roc_auc_score

In [16]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [17]:
X = df.drop('Survived',axis=1)
y = df['Survived']

In [18]:
X = pd.get_dummies(X,drop_first=True)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [21]:
grouped_feature_ls = []
correlated_groups = []
def correlation (dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr
corr_features=correlation (x_test, 0.9)
correlated_features = set(corr_features)
print(len(correlated_features))

0


In [22]:
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=True,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)

In [ ]:
sfs1=sfs1.fit(np.array(x_train.fillna(0)),y_train)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   27.6s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  1.7min
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  3.9min
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:  6.9min
[Parallel(n_jobs=1)]: Done 686 out of 686 | elapsed:  7.3min finished

[2025-08-13 09:27:51] Features: 1/5 -- score: 0.765194136612553[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   30.2s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  1.7min
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  3.9min
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:  6.8min
[Parallel(n_jobs=1)]: Done 685 out of 685 | elapsed:  7.1min finished

[2025-08-13 09:34:58] Features: 2/5 -- score: 0.832223720877483[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   23.7s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  1.6min
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  3.6min
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:  6.6min


In [ ]:
print('best accracy score',sfs1.k_score_)
print('best subset(indices)',sfs1.k_feature_idx_)
print('Best subset (corresponding names):', sfs1.k_feature_names_)

In [ ]:
sfs2=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=False,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)

In [ ]:
print('best accracy score',sfs2.k_score_)
print('best subset(indices)',sfs2.k_feature_idx_)
print('Best subset (corresponding names):', sfs2.k_feature_names_)

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
efs=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=5)
efs=efs.fit(x_train,y_train)
print('best accuracy score',efs.best_score_)
print('best subset(indices)',efs.best_idx_)
print('Best subset (corresponding names):', efs.best_feature_names_)

In [ ]:
efs_results = pd.DataFrame.from_dict(efs.get_metric_dict()).T
efs_results.sort_values(by='avg_score', ascending=True, inplace=True)
efs_results

In [ ]:
rfc=RandomForestClassifier(random_state=0,criterion='gini')